In [39]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from scipy.interpolate import interp1d #pre written interpolation function
from statsmodels.base.model import GenericLikelihoodModel
from scipy import stats #for kernel function

In [40]:
#TODO:

#debug value function
#debug kappa function

#paramterize cost function (for different specs)

#implement class using statsmodles
#iterate AM style

The code below references the following two sources.

John Rust's website:
https://editorialexpress.com/jrust/nfxp.html

Victor Aguirregabiria and Pedro Mira's website:
http://individual.utoronto.ca/vaguirre/wpapers/program_code_survey_joe_2008.html

Victor Aguirregabiria and Pedro Mira's 2002 paper
https://www.jstor.org/stable/3082006


In [41]:
#fix the bus .dat because Aguirregabiria and Mira hate everyone
data = np.fromfile('bus1234.dat')
data = data.reshape(len(data)/6,6)
data = pd.DataFrame(data,columns=['id','group','year','month','replace','miles'])

#save to .csv so other people don't need to be confused
data.to_csv("bus1234.csv")

#switch to date time for ease 
data['date'] = pd.to_datetime(data[['year', 'month']].assign(Day=1))
data = data[['id','group','date','replace','miles']]

#lag date
date_lag = data.copy()
date_lag['date'] = date_lag['date'] - pd.DateOffset(months=1)
data = data.merge(date_lag, how='left', on=['id','group','date'] , suffixes=('','_next'))
data = data.dropna()

print data.max()
print data.min()

id                              162
group                        530875
date            1985-04-01 00:00:00
replace                           1
miles                        388254
replace_next                      1
miles_next                   388254
dtype: object
id                               59
group                            50
date            1975-09-01 00:00:00
replace                           0
miles                             0
replace_next                      0
miles_next                        0
dtype: object


In [42]:
#constants
BETA = .97
GAMMA = .5772 #euler's constant

#size of step in discretization
STEP = 3000

In [43]:
def np_miles_pdf(i_obs, x_obs, x_next):
    """non-parametric estimation of mileage pdf.
    in other words, the probability of jumping dx states
    next period
    
    this corresponds to pdfdx in AM's code"""
    
    #figure out max number of steps
    dx = (1-i_obs)*(x_next - x_obs) + i_obs*x_obs 
    discrete = (dx/STEP).astype(int)
    
    #make a dataframe with the discrete 'jumps' in mileage
    frame = np.array([discrete, i_obs]).transpose()
    frame = pd.DataFrame(frame, columns=('dx','i'))

    #non-parametrically group jumps to make pdf
    pdfdx = frame.groupby('dx')
    pdfdx = pdfdx.count()/pdfdx.count().sum()
    return  np.array(pdfdx)

In [44]:
def miles_pdf(i_obs, x_obs, x_next):
    """estimation of mileage pdf following AM using the
    kernel function
    
    this corresponds to pdfdx in AM's code"""
    
    #figure out max number of steps
    dx = (1-i_obs)*(x_next - x_obs) + i_obs*x_obs 
    
    #number of states
    dx_states = np.arange(dx.min(),dx.max()  +1 ,STEP)
    
    #use kernel groups to make pdf
    kernel1 = stats.gaussian_kde(dx, bw_method='silverman')
    pdfdx = kernel1(dx_states)
    
    return np.array([pdfdx/pdfdx.sum()]).transpose()


MILES_PDF = miles_pdf(data['replace'], data['miles'], data['miles_next'])

In [45]:
def transition_1(i_obs, x_obs , x_next):
    """calculate transitions probabilities,
    non-parametrically
    
    this corresponds to fmat1 and fmat2 in AM's code"""
    
    #transitions when i=1
    num_states = (x_obs.max()/STEP).astype(int) + 1
    pdfdx = miles_pdf(i_obs, x_obs, x_next).transpose()
    
    #zero probability of transitioning to large states
    zeros = np.zeros( (num_states,num_states-pdfdx.shape[1]) )
    
    #transitioning to first state and 'jumping' dx states
    fmat1 = np.tile(pdfdx,(num_states,1))
    fmat1 = np.concatenate( (fmat1, zeros), axis=1 )

    return fmat1

FMAT1 = transition_1(data['replace'], data['miles'],data['miles_next'])

In [46]:
def transition_0(i_obs, x_obs , x_next):
    """calculate transitions probabilities,
    non-parametrically
    
    this corresponds to fmat1 and fmat2 in AM's code"""
    
    
    num_states = (x_obs.max()/STEP).astype(int) + 1
    pdfdx = miles_pdf(i_obs, x_obs, x_next).transpose()
    
    #initialize fmat array, transitions when i=0
    end_zeros = np.zeros((1,num_states -pdfdx.shape[1]))
    fmat0 = np.concatenate( (pdfdx,end_zeros), axis=1 )

    for row in range(1,num_states):
        
        #this corresponds to colz i think
        cutoff = (num_states - row - pdfdx.shape[1] )
        
        #case 1 far enough from the 'end' of the matrix
        if cutoff >= 0:
            start_zeros = np.zeros((1,row))
            end_zeros = np.zeros((1,num_states -pdfdx.shape[1] - row))
            fmat_new = np.concatenate( (start_zeros,pdfdx,end_zeros), axis=1 )
            fmat0 = np.concatenate((fmat0,fmat_new))
       
        #case 2, too far from the end and need to adjust probs
        else:
            pdf_adj = pdfdx[:,0:cutoff]
            pdf_adj = pdf_adj/pdf_adj.sum(axis=1)
            
            start_zeros = np.zeros((1,row))
            fmat_new = np.concatenate( (start_zeros,pdf_adj), axis=1 )
            fmat0 = np.concatenate((fmat0,fmat_new))


    return fmat0

FMAT0 = transition_0(data['replace'],data['miles'],data['miles_next'])

PR_TRANS = FMAT0, FMAT1

In [47]:
def np_initial_pr(i_obs, x_obs):
    """truly non-parametric
    initial the probability of view a given state
    May delete later"""
    
    num_states = ( x_obs.max()/STEP).astype(int) + 1
    discrete = (x_obs/STEP).astype(int)
    
    df = np.array([discrete, i_obs]).transpose()
    df = pd.DataFrame(df, columns=('x','i'))
    pr_obs = df.groupby('x')
    pr_obs = pr_obs.sum()/(1.*pr_obs.count())

    return  np.array(pr_obs)

PR_OBS = np_initial_pr(data['replace'], data['miles'])

In [48]:
def initial_pr(i_obs, x_obs, d=0):
    """initial the probability of view a given state following AM.
    Seems like it just involves logit to predict
    
    Third arugment involves display"""
    
    X = np.array([x_obs, x_obs**2, x_obs**3]).transpose()
    X = sm.add_constant(X)
    
    model = sm.Logit(i_obs,X)
    fit = model.fit(disp=d)
    if d: print fit.summary()
    
    states = np.arange(x_obs.min(),x_obs.max(), STEP)
    
    states = np.array([states, states**2, states**3]).transpose()
    states = sm.add_constant(states)
    
    return fit.predict(states)

PR_OBS = initial_pr(data['replace'], data['miles'], d=1)

Optimization terminated successfully.
         Current function value: 0.036201
         Iterations 14
                           Logit Regression Results                           
Dep. Variable:                replace   No. Observations:                 8156
Model:                          Logit   Df Residuals:                     8153
Method:                           MLE   Df Model:                            2
Date:                Fri, 11 Jan 2019   Pseudo R-squ.:                  0.1671
Time:                        16:55:54   Log-Likelihood:                -295.26
converged:                       True   LL-Null:                       -354.51
                                        LLR p-value:                 1.853e-26
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -17.3136      4.188     -4.134      0.000     -25.522      -9.105
x1             0.0001   5.67

In [55]:
def hm_value(theta1, cost, i_obs, x_obs, pr_obs, pr_trans):
    """calculate value function using hotz miller approach"""
    
    #set up matrices, transition is deterministic
    trans0, trans1 = pr_trans
    
    #should probably make these class parameters
    num_states = ( x_obs.max()/STEP).astype(int) + 1
    x = np.arange(x_obs.min(),x_obs.max(), STEP)
    
    
    #calculate value function for all state
    pr_tile = np.tile( pr_obs.reshape(num_states,1), (1,num_states))
    
    denom = (np.identity(num_states) - BETA*(1-pr_tile)*trans0 - BETA*trans1*pr_tile)
    
    numer = ( (1-pr_obs)*(theta1*x  + GAMMA - np.log(1-pr_obs)) + 
                 pr_obs*(cost+ GAMMA - np.log(pr_obs) ) )
    
    value = np.linalg.inv(denom).dot(numer)
    return value


VALUE = hm_value(-.004, -10.47, data['replace'], data['miles'], PR_OBS, PR_TRANS)

In [56]:
print VALUE

[-13445.40200143 -13691.26048343 -13935.21164651 -14177.11334271
 -14416.81817434 -14654.25086443 -14889.3592585  -15122.0896604
 -15352.46089279 -15580.59438957 -15806.51037108 -16030.03665223
 -16250.98974167 -16469.27012947 -16684.81982154 -16897.58774896
 -17107.52264362 -17314.56875186 -17518.66693187 -17719.795545
 -17917.97778196 -18113.18897093 -18305.33202884 -18494.28366765
 -18679.93431319 -18862.20955501 -19041.0568086  -19216.42977061
 -19388.28522956 -19556.5921755  -19721.33026153 -19882.45963239
 -20039.91768361 -20193.64600045 -20343.61397034 -20489.83125262
 -20632.32570577 -20771.11765924 -20906.21141066 -21037.616854
 -21165.39152432 -21289.63541299 -21410.45663965 -21527.94789635
 -21642.20912143 -21753.40801192 -21861.74985818 -21967.4046146
 -22070.56479772 -22171.46408779 -22270.34047249 -22367.41027367
 -22462.72694126 -22556.27777874 -22648.21624037 -22738.80423297
 -22828.26744057 -22916.73340195 -23004.2379491  -23090.7922929
 -23176.46583257 -23261.28665223

In [57]:
def hm_prob(theta1, cost, i_obs, x_obs, pr_obs, pr_trans):
    """calculate kappa using value function"""
    
    value = hm_value(theta1, cost, i_obs, x_obs, pr_obs, pr_trans)
    trans0, trans1 = pr_trans
    
    #should probably make these class parameters
    num_states = ( x_obs.max()/STEP).astype(int) + 1
    x = np.arange(x_obs.min(),x_obs.max(), STEP)
    
    delta1 = np.exp( cost + BETA*trans1.dot(value))
    delta0 = np.exp( x*theta1 + BETA*trans0.dot(value) )
    
    
    return delta1/(delta1+delta0)


hm_prob(-.004, -10.47, data['replace'], data['miles'], PR_OBS, PR_TRANS).shape

/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in divide
  from ipykernel import kernelapp as app


(130,)

In [58]:
from scipy.optimize import minimize

class HotzMiller():
    """class for estimating the values of R and theta
    using the Hotz Miller routine and the helper functions
    above"""
    
    def __init__(self, i, x, x_next):
        
        #transitions
        self.pr_obs = initial_pr(i, x)
        self.trans =  transition_1(i,x,x_next), transition_1(i,x,x_next)
        
        #should probably make these class parameters
        self.num_states = ( x.max()/STEP).astype(int) + 1
        self.states = np.arange(x.min(),x.max(), STEP)
        
        #data
        self.x = x
        self.x_next = x
        self.i = i
        
        #parameters
        self.theta1 = 0
        self.R = 0
        
        
    def likelihood(self, params): 
        theta1, R = params
        
        # Input our data into the model
        i = self.i
        x = (self.x/STEP).astype(int)
        
        #set up hm state pr
        prob = hm_prob(theta1, R, self.i, self.x, self.pr_obs, self.trans).transpose()
        prob = interp1d(self.states, prob, fill_value="extrapolate")
        
        log_likelihood = (1-i)*np.log(1-prob(x)) + i*np.log(prob(x))
        
        return -log_likelihood.sum()
    
    
    def fit(self):
        result = minimize(self.likelihood, [-.004,-10], method = 'Nelder-Mead', options={'disp': False})
        self.theta1, self.R = result.x

        
model_hm = HotzMiller(data['replace'], data['miles'],data['miles_next'])
model_hm.fit()


print '\n theta_1:%s, R:%s'%(round(model_hm.theta1,4) , round(model_hm.R,4))


 theta_1:-0.004, R:-10.0


/home/erichschulman/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in divide
  from ipykernel import kernelapp as app
